In [1]:
print('simpal')

simpal


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict


In [3]:
BASE_DIR = r'D:\ECG_model\multiple lead\ecg_images_multilead'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 30

In [4]:
image_paths = []
labels = []

for cls in range(1, 10):  # class_1 → class_9
    class_dir = os.path.join(BASE_DIR, f"class_{cls}")
    for f in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, f))
        labels.append(cls - 1)  # 0–8

image_paths = np.array(image_paths)
labels = np.array(labels)


In [5]:
ecg_map = defaultdict(list)

for i, path in enumerate(image_paths):
    ecg_id = os.path.basename(path).split("_win")[0]
    ecg_map[ecg_id].append(i)

ecg_ids = list(ecg_map.keys())

train_ecg, temp_ecg = train_test_split(ecg_ids, test_size=0.3, random_state=42)
val_ecg, test_ecg = train_test_split(temp_ecg, test_size=0.5, random_state=42)


In [6]:
def collect(ecg_list):
    idx = []
    for e in ecg_list:
        idx.extend(ecg_map[e])
    return idx

train_idx = collect(train_ecg)
val_idx   = collect(val_ecg)
test_idx  = collect(test_ecg)


In [7]:
def stage1_label(l):
    return 0 if l == 0 else 1   # 0=Normal, 1=Abnormal

y_stage1 = np.array([stage1_label(l) for l in labels])


In [8]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img, label


In [9]:
X_train, y_train = image_paths[train_idx], y_stage1[train_idx]
X_val,   y_val   = image_paths[val_idx],   y_stage1[val_idx]
X_test,  y_test  = image_paths[test_idx],  y_stage1[test_idx]


In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.map(load_image).shuffle(1000).batch(BATCH_SIZE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.map(load_image).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.map(load_image).batch(BATCH_SIZE)


In [11]:
base = EfficientNetB0(include_top=False, weights="imagenet",
                      input_shape=(224,224,3))
base.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(2, activation="softmax")(x)

stage1_model = models.Model(inputs, outputs)


In [12]:
stage1_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [13]:
stage1_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True)]
)


Epoch 1/30
1086/1086 [==============================] - 145s 96ms/step - loss: 0.3945 - accuracy: 0.8400 - val_loss: 0.3484 - val_accuracy: 0.8597
Epoch 2/30
1086/1086 [==============================] - 103s 95ms/step - loss: 0.3045 - accuracy: 0.8867 - val_loss: 0.3371 - val_accuracy: 0.8660
Epoch 3/30
1086/1086 [==============================] - 105s 96ms/step - loss: 0.2940 - accuracy: 0.8906 - val_loss: 0.3346 - val_accuracy: 0.8659
Epoch 4/30
1086/1086 [==============================] - 105s 97ms/step - loss: 0.2892 - accuracy: 0.8892 - val_loss: 0.3293 - val_accuracy: 0.8675
Epoch 5/30
1086/1086 [==============================] - 103s 95ms/step - loss: 0.2835 - accuracy: 0.8916 - val_loss: 0.3297 - val_accuracy: 0.8672
Epoch 6/30
1086/1086 [==============================] - 103s 95ms/step - loss: 0.2812 - accuracy: 0.8929 - val_loss: 0.3267 - val_accuracy: 0.8684
Epoch 7/30
1086/1086 [==============================] - 104s 95ms/step - loss: 0.2780 - accuracy: 0.8946 - val_loss: 0

In [14]:
win_preds = []
win_true  = []

for imgs, labs in test_ds:
    preds = stage1_model.predict(imgs, verbose=0)
    win_preds.extend(np.argmax(preds, axis=1))
    win_true.extend(labs.numpy())


In [15]:
ecg_preds = defaultdict(list)
ecg_true = {}

for path, p, t in zip(X_test, win_preds, win_true):
    ecg_id = os.path.basename(path).split("_win")[0]
    ecg_preds[ecg_id].append(p)
    ecg_true[ecg_id] = t


In [16]:
final_pred = []
final_true = []

for e in ecg_preds:
    final_pred.append(max(set(ecg_preds[e]), key=ecg_preds[e].count))
    final_true.append(ecg_true[e])


In [17]:
print("STAGE-1 ECG Accuracy:",
      accuracy_score(final_true, final_pred))
print(classification_report(final_true, final_pred, digits=3))


STAGE-1 ECG Accuracy: 0.8515037593984962
              precision    recall  f1-score   support

           0      0.875     0.154     0.262        91
           1      0.851     0.995     0.917       441

    accuracy                          0.852       532
   macro avg      0.863     0.575     0.590       532
weighted avg      0.855     0.852     0.805       532



In [18]:
abnormal_ecgs = [e for e in ecg_preds if ecg_true[e] == 1]


In [19]:
stage2_paths = []
stage2_labels = []

for e in abnormal_ecgs:
    for idx in ecg_map[e]:
        stage2_paths.append(image_paths[idx])
        stage2_labels.append(labels[idx] - 1)  # shift (remove normal)


In [20]:
stage2_paths = np.array(stage2_paths)
stage2_labels = np.array(stage2_labels)
NUM_STAGE2_CLASSES = len(np.unique(stage2_labels))


In [21]:
X_tr, X_te, y_tr, y_te = train_test_split(
    stage2_paths, stage2_labels,
    test_size=0.3, random_state=42, stratify=stage2_labels
)


In [22]:
train2_ds = tf.data.Dataset.from_tensor_slices((X_tr, y_tr))
train2_ds = train2_ds.map(load_image).shuffle(1000).batch(BATCH_SIZE)

test2_ds = tf.data.Dataset.from_tensor_slices((X_te, y_te))
test2_ds = test2_ds.map(load_image).batch(BATCH_SIZE)


In [23]:
base2 = EfficientNetB0(include_top=False, weights="imagenet",
                       input_shape=(224,224,3))
base2.trainable = False

inputs = layers.Input(shape=(224,224,3))
x = base2(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
outputs = layers.Dense(NUM_STAGE2_CLASSES, activation="softmax")(x)

stage2_model = models.Model(inputs, outputs)


In [ ]:
stage2_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
stage2_model.fit(
    train2_ds,
    epochs=EPOCHS,
    callbacks=[tf.keras.callbacks.EarlyStopping(
        monitor="loss", patience=5, restore_best_weights=True)]
)

Epoch 1/30
136/136 [==============================] - 15s 81ms/step - loss: 2.2375 - accuracy: 0.1902
Epoch 2/30
136/136 [==============================] - 11s 80ms/step - loss: 1.9020 - accuracy: 0.3011
Epoch 3/30
136/136 [==============================] - 11s 80ms/step - loss: 1.7689 - accuracy: 0.3566
Epoch 4/30
136/136 [==============================] - 11s 80ms/step - loss: 1.6797 - accuracy: 0.3913
Epoch 5/30
136/136 [==============================] - 11s 80ms/step - loss: 1.6143 - accuracy: 0.4203
Epoch 6/30
136/136 [==============================] - 11s 81ms/step - loss: 1.5665 - accuracy: 0.4358
Epoch 7/30
136/136 [==============================] - 11s 81ms/step - loss: 1.5219 - accuracy: 0.4582
Epoch 8/30
136/136 [==============================] - 11s 80ms/step - loss: 1.4799 - accuracy: 0.4784
Epoch 9/30
136/136 [==============================] - 11s 80ms/step - loss: 1.4489 - accuracy: 0.4848
Epoch 10/30
136/136 [==============================] - 11s 81ms/step - loss: 1.423

In [47]:
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict
import numpy as np
import os

# ===============================
# STAGE-2 WINDOW-LEVEL PREDICTION
# ===============================
win_preds_2 = []
win_true_2  = []
win_paths_2 = []

for imgs, labs in test2_ds:
    preds = stage2_model.predict(imgs, verbose=0)
    win_preds_2.extend(np.argmax(preds, axis=1))
    win_true_2.extend(labs.numpy())

# collect corresponding paths
win_paths_2 = X_te  # SAME ORDER as test2_ds creation

# ===============================
# AGGREGATE TO ECG-LEVEL
# ===============================
ecg_preds_2 = defaultdict(list)
ecg_true_2  = {}

for path, p, t in zip(win_paths_2, win_preds_2, win_true_2):
    ecg_id = os.path.basename(path).split("_win")[0]
    ecg_preds_2[ecg_id].append(p)
    ecg_true_2[ecg_id] = t

final_pred_2 = []
final_true_2 = []

for e in ecg_preds_2:
    # majority voting (same as Stage-1)
    final_pred_2.append(
        max(set(ecg_preds_2[e]), key=ecg_preds_2[e].count)
    )
    final_true_2.append(ecg_true_2[e])

final_pred_2 = np.array(final_pred_2)
final_true_2 = np.array(final_true_2)

# ===============================
# FINAL STAGE-2 ECG REPORT
# ===============================
print("STAGE-2 ECG Accuracy:",
      accuracy_score(final_true_2, final_pred_2))

print(classification_report(
    final_true_2,
    final_pred_2,
    digits=3
))


STAGE-2 ECG Accuracy: 0.6100917431192661
              precision    recall  f1-score   support

           0      0.729     0.775     0.752        80
           1      0.535     0.479     0.505        48
           2      0.600     0.643     0.621        14
           3      0.530     0.892     0.665       120
           4      0.724     0.404     0.519        52
           5      0.640     0.320     0.427        50
           6      0.727     0.414     0.527        58
           7      1.000     0.286     0.444        14

    accuracy                          0.610       436
   macro avg      0.686     0.527     0.557       436
weighted avg      0.646     0.610     0.592       436

